<a href="https://colab.research.google.com/github/EHOTik/recognition-of-DNA-secondary-structures/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colaboratory!%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 4.3MB/s 
     |████████████████████████████████| 901kB 34.1MB/s 
     |████████████████████████████████| 3.3MB 36.5MB/s 


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
# import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
data = pd.read_csv('train_data_new.csv', delimiter=',',names=['chrom', 'short_seq', 'target'])
batch_1 = data[data['short_seq'].notnull()]

In [ ]:
# For DistilBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-large-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenized = batch_1['short_seq'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

In [ ]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = batch_1['target']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(test_features, test_labels) #bert-large

0.6028985507246377

In [ ]:
lr_clf.score(test_features, test_labels) #bert-base

0.6028985507246377

In [ ]:
!pip install fair-esm

In [ ]:
import pandas as pd
data = pd.read_csv('train_data_new.csv', delimiter=',',names=['chrom', 'short_seq', 'target'])
batch_1 = data[data['short_seq'].notnull()]

In [ ]:
import torch
import esm

# Load ESM-1b model
model, alphabet = esm.pretrained.esm1_t6_43M_UR50S()
batch_converter = alphabet.get_batch_converter()

In [ ]:
# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
# data = [("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
#         ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),]
# batch_labels, batch_strs, batch_tokens = batch_converter(data)

In [ ]:
arr = [tuple(elem[:2]) for elem in batch_1.values.tolist()]
i = 0
data = arr[i:i+4]

In [ ]:
sequence_representations = []
for k in range(len(arr)):
    data = arr[k]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6], return_contacts=True)

    token_representations = results["representations"][6]

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    for i, (_, seq) in enumerate(data):
        sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))
    print(len(sequence_representations))

ValueError: ignored

In [ ]:
len(sequence_representations)

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 7.8MB/s 
     |████████████████████████████████| 225kB 48.3MB/s 
     |████████████████████████████████| 122kB 56.0MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 3.3MB 42.6MB/s 
     |████████████████████████████████| 8.2MB 40.0MB/s 
     |████████████████████████████████| 2.3MB 31.0MB/s 
     |████████████████████████████████| 1.2MB 44.3MB/s 
     |████████████████████████████████| 1.8MB 40.5MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 112kB 55.2MB/s 
     |████████████████████████████████| 245kB 56.2MB/s 
     |████████████████████████████████| 174kB 55.0MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 112kB 56.0MB/s 
     |████████████████████████████████| 4.2MB 38.5MB/s 
     |████████████████████████████████| 901kB 47.3MB/s 
     |████████████████████████████████| 102kB 10.7MB/

In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=4,
    weight=[1, 0.5, 1, 2]
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_p

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
train_data = [
    ["Aragorn was the heir of Isildur", 1.0],
    ["Frodo was the heir of Isildur", 0.0],
    ["Pippin is stronger than Merry", 0.3],
]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

In [ ]:
model.train_model(df_train)
model.eval_model(df_test)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


({'eval_loss': 1.4020287990570068, 'mcc': 0.0},
 array([[-0.0791626 ,  0.14782715, -0.019104  , -0.14038086],
        [-0.07476807,  0.14916992, -0.01702881, -0.14074707],
        [-0.07568359,  0.14733887, -0.01424408, -0.13549805]]),
 [[1.0, 0.0, 0.3]])

In [ ]:
predictions, raw_outputs = model.predict(["Sam was a Wizard"])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_outputs

array([[-0.07470703,  0.14489746, -0.00828552, -0.14050293]])

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
train_data = [
    ["Aragorn was the heir of Isildur", 1],
    ["Frodo was the heir of Isildur", 0],
]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

# Preparing eval data
eval_data = [
    ["Theoden was the king of Rohan", 1],
    ["Merry was the king of Rohan", 0],
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

# Create a ClassificationModel
model_ = ClassificationModel(
    "roberta", "roberta-base", args=model_args
)

# Train the model
model_.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model_.eval_model(eval_df)

# Make predictions with the model
predictions, raw_outputs = model_.predict(["Sam was a Wizard"])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_p

  0%|          | 0/2 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 0, 'tn': 1, 'fp': 0, 'fn': 1, 'auroc': 1.0, 'auprc': 1.0, 'eval_loss': 0.6956483125686646}
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 27.9MB/s 
     |████████████████████████████████| 225kB 52.6MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 1.8MB 36.9MB/s 
     |████████████████████████████████| 3.3MB 40.8MB/s 
     |████████████████████████████████| 1.2MB 39.5MB/s 
     |████████████████████████████████| 122kB 56.6MB/s 
     |████████████████████████████████| 2.3MB 48.7MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 8.2MB 36.2MB/s 
     |████████████████████████████████| 112kB 54.9MB/s 
     |████████████████████████████████| 245kB 27.0MB/s 
     |████████████████████████████████| 174kB 52.8MB/s 
     |████████████████████████████████| 133kB 56.8MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 901kB 44.8MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 4.2MB 39.0MB

In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import numpy as np
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data

data = pd.read_csv('train_data_new.csv', delimiter=',',names=['chrom', 'text', 'labels'])
data_1 = data[data['text'].notnull()]
df = data_1.drop(['chrom'], axis='columns')

msk = np.random.rand(len(df)) < 0.8
train_df = df[msk]
test_df = df[~msk]

In [ ]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=10)


# Create a ClassificationModel
model_1 = ClassificationModel(
    "roberta", "roberta-base", args=model_args
)

# Train the model
model_1.train_model(train_df)

# Evaluate the model
result_1, model_outputs_1, wrong_predictions_1 = model_1.eval_model(test_df)

INFO:filelock:Lock 139807272000848 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

INFO:filelock:Lock 139807272000848 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
INFO:filelock:Lock 139809964576272 acquired on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock


Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

INFO:filelock:Lock 139809964576272 released on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of R

...

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2471 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/309 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.8124130097803833, 'tp': 778, 'tn': 1469, 'fp': 31, 'fn': 193, 'auroc': 0.921596979059389, 'auprc': 0.9033159221306801, 'eval_loss': 0.36888702609500934}


In [ ]:
result_1

{'auprc': 0.9033159221306801,
 'auroc': 0.921596979059389,
 'eval_loss': 0.36888702609500934,
 'fn': 193,
 'fp': 31,
 'mcc': 0.8124130097803833,
 'tn': 1469,
 'tp': 778}

In [ ]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)


# Create a ClassificationModel
model_2 = ClassificationModel(
    "camembert", "camembert-base", args=model_args,
    use_cuda = False
)

INFO:filelock:Lock 139761474256144 acquired on /root/.cache/huggingface/transformers/f459e43c5ebb871abbf9209195563bff6a11547fd9532047739667c394833221.e23d229c54bcc6f67d337b8b2dd111b0e3dc01fa854bfecd3efdeb8c955749e6.lock


Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

INFO:filelock:Lock 139761474256144 released on /root/.cache/huggingface/transformers/f459e43c5ebb871abbf9209195563bff6a11547fd9532047739667c394833221.e23d229c54bcc6f67d337b8b2dd111b0e3dc01fa854bfecd3efdeb8c955749e6.lock
INFO:filelock:Lock 139761497867152 acquired on /root/.cache/huggingface/transformers/7e23f45751ad1fed420ca9f03bb37a279dc98a56c75bf25e671129237e2c893c.ee4d4253e08a7cf9697c0671fd8f022483dbf586691a7b32ead55493a34d72b2.lock


Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

INFO:filelock:Lock 139761497867152 released on /root/.cache/huggingface/transformers/7e23f45751ad1fed420ca9f03bb37a279dc98a56c75bf25e671129237e2c893c.ee4d4253e08a7cf9697c0671fd8f022483dbf586691a7b32ead55493a34d72b2.lock
Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weig

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

INFO:filelock:Lock 139758805139856 released on /root/.cache/huggingface/transformers/dbcb433aefd8b1a136d029fe2205a5c58a6336f8d3ba20e6c010f4d962174f5f.160b145acd37d2b3fd7c3694afcf4c805c2da5fd4ed4c9e4a23985e3c52ee452.lock
INFO:filelock:Lock 139758799385168 acquired on /root/.cache/huggingface/transformers/84c442cc6020fc04ce266072af54b040f770850f629dd86c5951dbc23ac4c0dd.8fd2f10f70e05e6bf043e8a6947f6cdf9bb5dc937df6f9210a5c0ba8ee48e959.lock


Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

INFO:filelock:Lock 139758799385168 released on /root/.cache/huggingface/transformers/84c442cc6020fc04ce266072af54b040f770850f629dd86c5951dbc23ac4c0dd.8fd2f10f70e05e6bf043e8a6947f6cdf9bb5dc937df6f9210a5c0ba8ee48e959.lock
/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:346: UserWarning: use_multiprocessing automatically disabled as camembert fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"


In [ ]:
# Train the model
model_2.train_model(train_df)

# Evaluate the model
result_2, model_outputs_2, wrong_predictions_2 = model_2.eval_model(test_df)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_camembert_128_2_2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1250 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of camembert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2427 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_camembert_128_2_2


Running Evaluation:   0%|          | 0/304 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.8516279232176585, 'tp': 819, 'tn': 1438, 'fp': 53, 'fn': 117, 'auroc': 0.9693502897728251, 'auprc': 0.9586263553995327, 'eval_loss': 0.22496185854910627}


In [ ]:
result_2 

{'auprc': 0.9586263553995327,
 'auroc': 0.9693502897728251,
 'eval_loss': 0.22496185854910627,
 'fn': 117,
 'fp': 53,
 'mcc': 0.8516279232176585,
 'tn': 1438,
 'tp': 819}

In [ ]:
result_2 

{'auprc': 0.9610943022749521,
 'auroc': 0.9679574133542979,
 'eval_loss': 0.3218176423284871,
 'fn': 221,
 'fp': 25,
 'mcc': 0.7991677801627991,
 'tn': 1485,
 'tp': 773}

In [ ]:
!rm -rf outputs

In [ ]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)


# Create a ClassificationModel
model_3 = ClassificationModel(
    "bert", "bert-base-uncased", args=model_args, 
    use_cuda = False
)

# Train the model
model_3.train_model(train_df)

# Evaluate the model
result_3, model_outputs_3, wrong_predictions_3 = model_3.eval_model(test_df)

INFO:filelock:Lock 140520524680208 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

INFO:filelock:Lock 140520524680208 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
INFO:filelock:Lock 140520524679312 acquired on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

INFO:filelock:Lock 140520524679312 released on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you exp

...

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2497 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/313 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 0, 'tn': 1511, 'fp': 0, 'fn': 986, 'auroc': 0.4999120714489954, 'auprc': 0.3948321370988714, 'eval_loss': 0.6704996854733354}


In [ ]:
result_3

{'auprc': 0.3948321370988714,
 'auroc': 0.4999120714489954,
 'eval_loss': 0.6704996854733354,
 'fn': 986,
 'fp': 0,
 'mcc': 0.0,
 'tn': 1511,
 'tp': 0}

In [ ]:
!rm -rf outputs

In [ ]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=5)


# Create a ClassificationModel
model_4 = ClassificationModel(
    "bert", "bert-large", args=model_args
)

# Train the model
model_4.train_model(train_df)

# Evaluate the model
result_4, model_outputs_4, wrong_predictions_4 = model_4.eval_model(test_df)

In [ ]:
result_4

In [ ]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)


# Create a ClassificationModel
model_3 = ClassificationModel(
    "bert", "bert-base-uncased", args=model_args, 
    use_cuda = False
)

INFO:filelock:Lock 140289603537936 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

INFO:filelock:Lock 140289603537936 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
INFO:filelock:Lock 140289600866448 acquired on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

INFO:filelock:Lock 140289600866448 released on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you exp

...

INFO:filelock:Lock 140289600943824 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
INFO:filelock:Lock 140289594804048 acquired on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

INFO:filelock:Lock 140289594804048 released on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
INFO:filelock:Lock 140289600943824 acquired on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

INFO:filelock:Lock 140289600943824 released on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
